# Loading an "External System" Transaction XML into LUSID

The purpose of this notebook  is to demonstrate how a user could load a transaction XML file from another "External System" into LUSID. The notebook will walk through the following steps: <br>

(1) Create a portfolio in LUSID. <br>
(2) Load a transaction XML into a Pandas DataFrame. <br>
(3) Translate the data in DataFrame into data types which LUSID will recognise (e.g. transform "strings" to "datetimes"). <br>
(4) Use client identifiers to create placeholder LUIDS (these can be enriched at a later date once we receive the "External System" instrument master file). <br>
(5) Load the transaction into LUSID using the Transactions API. <br><br>


<div>
<img src="img/python_env.png"/>
</div>

## Setup: Load libraries and create our LUSID API client


In [1]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
import globalfund as global_fund_tools
import cocoon as cocoon_tools

# Import Libraries
import pprint
from datetime import datetime, timedelta, time, date
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
import json
import xml.etree.ElementTree as ET
import io

# Authenticate our user and create our API client
lusid_client = import_data.authenticate_secrets()

print('LUSID Client Initialised')
print('LUSID version : ', lusid_client.metadata.get_lusid_versions().build_version)

LUSID Client Initialised
LUSID version :  0.5.3356.0


As part of the setup, we need to define a Scope. Scopes in LUSID are used to partition data. In this example, we will use the Scope "EXT_SYSTEM" to represent an external trading system. 

In [2]:
# define scope

scope = "EXT_SYSTEM"

## Step 1: Create a portfolio in LUSID

First we need a portfolio in LUSID. The "External System" transaction XML will be loaded into this portfolio. For this example, I manually created the portfolio in LUSID's front-end web-tool, but we could also use the [CreatePortfolios API](
https://www.lusid.com/docs/api/#operation/CreatePortfolio) to programmatically create portfolios in bulk.




<div>
<img src="img/create_portfolio.png" width="750"/>
</div>

In [3]:
# define portfolio variable for portfolio we just created

portfolio = "ext_port_1"

## Step 2: Scrape Transaction XML data into a Python DataFrame

Next we need to load the XML file into a Python dataframe. This will allow us to transform the data into a format LUSID will recognise.

In [4]:
# Load the XML file into memory

ext_system_file = os.path.join("ext_system_file.xml")

In [5]:
# Create a Python object from the XML file
# List out the individual tags which will need to be mapped

tree = ET.parse(ext_system_file)
root = tree.getroot()
list_tags = list(set([elem.tag for elem in root.iter()]))
for tag in list_tags:
    print(tag)

TRD_TRANS_TYPE
TRD_SETTLE_DATE
TRD_COUNTERPARTY
TRANSACTIONS
TRD_PRINCIPAL
PORT_NAME
TRD_NUMBER
TRD_LOCATION
TRD_TRADER
TRD_PRICE
INST_TYPE
INST_TICKER
TRD_TRADE_DATE
INST_GROUP
TRD_QUANTITY
INST_DESC
TRD_CURRENCY
TRD_TOUCH_COUNT
TRADE
CLIENT_ID


Create a list of the tags which we want to load into LUSID. For the purposes of this example, we will select a subset of fields from the sample XML file. In production, you may need to load more.

In [6]:
list_tags = [
    
    "TRD_TRANS_TYPE",
    "TRD_QUANTITY",
    "TRD_TRADE_DATE",
    "PORT_NAME",
    "INST_GROUP",
    "TRD_CURRENCY",
    "TRD_TOUCH_COUNT",
    "INST_DESC",
    "INST_TYPE",
    "TRD_PRINCIPAL",
    "TRD_TRADER",
    "TRD_PRICE",
    "TRD_COUNTERPARTY",
    "INST_TICKER",
    "CLIENT_ID",
    "TRD_SETTLE_DATE",
    "TRD_LOCATION",
    "TRD_NUMBER"
    
]

Iterate over the "External System" transaction XML and scrape the results into a Pandas DataFrame. We scrape the results into a DataFrame for two reasons:<br>

(1) We can use the DataFrame methods to transform the data into a format LUSID likes. <br>
(2) We can then use the SDKs to pass the DataFrame directly to LUSID<br>
<br>

In [7]:
# Iterate over XML and create a DataFrame

rows= []

for trade in root.iter('TRADE'):
    
    trade_list = []
    for tag in list_tags:
        
        try:       
            trade_list.append((trade.find(tag).text))
        except AttributeError:
            trade_list.append(None)

    rows.append({"TRD_TRANS_TYPE": trade_list[0], 
                 "TRD_QUANTITY": trade_list[1], 
                 "TRD_TRADE_DATE": trade_list[2],
                 "PORT_NAME": trade_list[3],
                 "INST_GROUP": trade_list[4],
                 "TRD_CURRENCY": trade_list[5],
                 "TRD_TOUCH_COUNT": trade_list[6],
                 "INST_DESC": trade_list[7],
                 "INST_TYPE": trade_list[8],
                 "TRD_PRINCIPAL": trade_list[9],
                 "TRD_TRADER": trade_list[10],
                 "TRD_PRICE": trade_list[11],
                 "TRD_COUNTERPARTY": trade_list[12],
                 "INST_TICKER": trade_list[13],
                 "CLIENT_ID": trade_list[14],
                 "TRD_SETTLE_DATE": trade_list[15],
                 "TRD_LOCATION": trade_list[16],
                 "TRD_NUMBER": trade_list[17]
                
                })
    
ext_system_df = pd.DataFrame(rows)

ext_system_df = ext_system_df.head(10)

Let's take a look at the DataFrame. The results look good. We see trade details - broker, security type, price etc. 

In [8]:
ext_system_df

,CLIENT_ID,INST_DESC,INST_GROUP,INST_TICKER,INST_TYPE,PORT_NAME,TRD_COUNTERPARTY,TRD_CURRENCY,TRD_LOCATION,TRD_NUMBER,TRD_PRICE,TRD_PRINCIPAL,TRD_QUANTITY,TRD_SETTLE_DATE,TRD_TOUCH_COUNT,TRD_TRADER,TRD_TRADE_DATE,TRD_TRANS_TYPE
0,EXT1YU6H5,INTERNATIONAL BUSINESS MACHINES CORP,EQUITY,IBM,COMMON_STOCK,TEST_PORT1,GS,USD,LONDON,0011223,142.99,7864450.000,55000.0000000000,8/20/2018,2,TEST_TRADER,8/16/2018,BUY
1,EXT1YU6H6,BANK OF AMERICA CORP,EQUITY,BAC,COMMON_STOCK,TEST_PORT1,JPM,USD,LONDON,0011224,28.99,2899000.000,100000.0000000000,9/20/2018,1,TEST_TRADER,9/16/2018,BUY
2,EXT1YU6H7,WALMART INC,EQUITY,WMT,COMMON_STOCK,TEST_PORT2,JPM,USD,LONDON,0011224,116.99,11699000,100000.0000000000,10/20/2018,1,TEST_TRADER,10/16/2018,SELL
3,EXT1YU6H8,3M CO,EQUITY,MMM,COMMON_STOCK,TEST_PORT2,MSUSA,USD,LONDON,0011224,155.99,11599000,100000.0000000000,5/20/2018,6,TEST_TRADER,5/16/2018,BUY


## Step 3: Translate the data into a LUSID-friendly format

The script above loads "Trade Date" and "Settle Date" as strings. LUSID needs these as datetimes.

In [9]:
ext_system_df['TRD_TRADE_DATE'] = pd.to_datetime(
    ext_system_df['TRD_TRADE_DATE'], format="%m/%d/%Y").apply(lambda x: pytz.utc.localize(x))

ext_system_df['TRD_SETTLE_DATE'] = pd.to_datetime(
    ext_system_df['TRD_SETTLE_DATE'], format="%m/%d/%Y").apply(lambda x: pytz.utc.localize(x))

In [10]:
ext_system_df

,CLIENT_ID,INST_DESC,INST_GROUP,INST_TICKER,INST_TYPE,PORT_NAME,TRD_COUNTERPARTY,TRD_CURRENCY,TRD_LOCATION,TRD_NUMBER,TRD_PRICE,TRD_PRINCIPAL,TRD_QUANTITY,TRD_SETTLE_DATE,TRD_TOUCH_COUNT,TRD_TRADER,TRD_TRADE_DATE,TRD_TRANS_TYPE
0,EXT1YU6H5,INTERNATIONAL BUSINESS MACHINES CORP,EQUITY,IBM,COMMON_STOCK,TEST_PORT1,GS,USD,LONDON,0011223,142.99,7864450.000,55000.0000000000,2018-08-20 00:00:00+00:00,2,TEST_TRADER,2018-08-16 00:00:00+00:00,BUY
1,EXT1YU6H6,BANK OF AMERICA CORP,EQUITY,BAC,COMMON_STOCK,TEST_PORT1,JPM,USD,LONDON,0011224,28.99,2899000.000,100000.0000000000,2018-09-20 00:00:00+00:00,1,TEST_TRADER,2018-09-16 00:00:00+00:00,BUY
2,EXT1YU6H7,WALMART INC,EQUITY,WMT,COMMON_STOCK,TEST_PORT2,JPM,USD,LONDON,0011224,116.99,11699000,100000.0000000000,2018-10-20 00:00:00+00:00,1,TEST_TRADER,2018-10-16 00:00:00+00:00,SELL
3,EXT1YU6H8,3M CO,EQUITY,MMM,COMMON_STOCK,TEST_PORT2,MSUSA,USD,LONDON,0011224,155.99,11599000,100000.0000000000,2018-05-20 00:00:00+00:00,6,TEST_TRADER,2018-05-16 00:00:00+00:00,BUY


## Step 4: Create placeholder LUIDs from "External System" client IDs

LUSID needs unique identifiers to upload transactions. These identifiers are called LUIDs. An instrument's LUID is generated when it is created in your instrument master inside LUSID.<br>

For the purposes of this example, we will use the client ID to create a placeholder LUID. This LUID can be enriched at a late date when we recieve the "External System" instrument file, which will contain details of public IDs and other static data. <br>





In [12]:
# Create a DataFrame containing the "External System" ID

instruments = ext_system_df[["CLIENT_ID"]]
instruments.loc[:,"Security Description"] = "Placeholder LUID - Enrichment Required"
instruments

,CLIENT_ID,Security Description
0,EXT1YU6H5,Placeholder LUID - Enrichment Required
1,EXT1YU6H6,Placeholder LUID - Enrichment Required
2,EXT1YU6H7,Placeholder LUID - Enrichment Required
3,EXT1YU6H8,Placeholder LUID - Enrichment Required


In [13]:
instrument_mapping_required = {
  'name': 'Security Description'
}

instrument_identifier_mapping = {
  'identifier_mapping': {
    'ClientInternal': 'CLIENT_ID',
  }
}

instrument_mapping_optional = None

In [14]:
response = cocoon_tools.load_instruments(
    client=lusid_client, 
    data_frame=instruments, 
    instrument_identifier_mapping=instrument_identifier_mapping, 
    instrument_mapping_required=instrument_mapping_required,
    instrument_mapping_optional=instrument_mapping_optional,
    property_columns=[], 
    scope='InstrumentProperties001')

prettyprint.instrument_response(response)

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,Placeholder LUID - Enrichment Required,EXT1YU6H8,LUID_0OM932XX


## Step 5: Load transactions into LUSID

Finally, we want to load the transactions into LUSID. This requires two steps.<br>

First, we need to resolve the client IDs to LUIDs.

In [15]:
transaction_identifier_mapping = {
  'identifier_mapping': {
      'ClientInternal': "CLIENT_ID",
      
  },
  'is_cash_with_currency': 'is_cash_with_currency'
}

In [16]:
ext_system_df["FUND"] = portfolio
ext_system_df["is_cash_with_currency"] = np.nan
ext_system_df["exchange_rate"] = 1

In [17]:
transactions_data = cocoon_tools.resolve_instruments(
    client=lusid_client,
    data_frame=ext_system_df,
    identifier_mapping=transaction_identifier_mapping)

current_time = datetime.now(pytz.UTC)

transactions_data.loc[
    transactions_data['resolvable'] == False].to_csv(
        './data/UnResolved{}-{}-{}-{}-{}-{}.csv'.format(
            'Transactions',
            current_time.year,
            current_time.month,
            current_time.day,
            current_time.hour,
            current_time.minute))

transactions_data = transactions_data.loc[transactions_data['resolvable'] == True]

Up to row 0


Now, once we have a Transaction file with LUIDs, we pass this to the Transactions API endpoint.

In [18]:
transaction_field_mapping_required = {
    "portfolio_code": "FUND",
    "transaction_id": "TRD_NUMBER",
    "transaction_type": 'TRD_TRANS_TYPE',
    "transaction_date": 'TRD_TRADE_DATE',
    "settlement_date": 'TRD_SETTLE_DATE',
    "units": "TRD_QUANTITY",
    "transaction_price.price": "TRD_PRICE",
    "total_consideration.amount": "TRD_PRINCIPAL",
    "total_consideration.currency": "TRD_CURRENCY",
    "transaction_currency": "TRD_CURRENCY"
    }

transaction_field_mapping_optional = {
    "exchange_rate": "exchange_rate"
}

In [19]:
transactions_data = transactions_data

responses = cocoon_tools.load_file_multiple_portfolios(
    client=lusid_client, 
    scope=scope, 
    data_frame=transactions_data,
    mapping_required=transaction_field_mapping_required, 
    mapping_optional=transaction_field_mapping_optional,
    source=scope,
    file_type="transaction")

for portfolio_code, response in responses.items():
    print ('\n')
    prettyprint.transactions_response(response, scope, portfolio_code)

data types don't match for column is_cash_with_currency it is string in LUSID and float64 in file
Updated is_cash_with_currency to object
Check for missing transaction properties complete


Transactions Successfully Upserted into Portfolio
Scope: EXT_SYSTEM
Code: portfolio
Transactions Effective From: 2018-05-16 00:00:00+00:00
Transactions Created On: 2019-10-07 15:36:52.995845+00:00



The transactions are now visible in the front-end LUSID tool.

<div>
<img src="img/read_transactions.png" width="950"/>
</div>